In [1]:
import pandas as pd
import numpy as np
# !pip install plotly --user
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import sys
sys.path.append("../../scripts/")
from importlib import reload
import TweetToVect2 as TW
import UserToVect as UVect

# !pip install dash --user

import dash
import dash_core_components as dcc
import dash_html_components as html

In [2]:
import tensorflow as tf

In [83]:
users_df = pd.read_pickle('../../datasets/users_vectores.pkl')
users_partidos = pd.read_csv('../../datasets/user_partido.csv')
users_partidos.columns=['user_id', 'user_name', 'partido', 'partido_original']
def unificar_partido(row):
    row.partido = row.partido if pd.isna(row.partido_original) else row.partido_original
    return row
users_partidos = users_partidos.apply(unificar_partido,axis=1)
users_partidos = users_partidos[users_partidos.user_id.isin(users_df.user_id)]
users_df.loc[users_partidos.user_id.values, 'partido'] = users_partidos.partido.values
users_df.loc[users_partidos.user_id.values, 'partido_original'] = users_partidos.partido_original.values

In [84]:
users_df.groupby('partido').count().user_id

partido
CONSENSO FEDERAL                                8
FRENTE DE IZQUIERDA Y DE LOS TRABAJADORES       2
FRENTE DE TODOS                              2499
JUNTOS POR EL CAMBIO                         1072
PARLAMENTARIO FEDERAL                           3
PARTIDO DE LA CONCORDIA SOCIAL                  1
PRODUCCIÓN Y TRABAJO                            1
PTS - FRENTE DE IZQUIERDA                     176
UNIDAD FEDERAL PARA EL DESARROLLO               4
Name: user_id, dtype: int64

In [85]:
users = users_df[users_df.partido.isin(['JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS', 'PTS - FRENTE DE IZQUIERDA'])]

In [86]:
sample_index = users[users.partido.isin(['JUNTOS POR EL CAMBIO', 'FRENTE DE TODOS'])].sample(100).user_id.values
sample_2  = users[users.partido.isin(['PTS - FRENTE DE IZQUIERDA'])].sample(100).user_id.values
df_to_plot = users[
    (users.user_id.isin(sample_index)).values |
    (pd.notna(users_df.partido_original)) |
    (users.user_id.isin(sample_2)).values
]
len(df_to_plot)

ValueError: operands could not be broadcast together with shapes (3747,) (86009,) 

In [71]:
vectores = df_to_plot.vector_user.values.tolist()
user_model = UVect.UserToVect()
distancias = user_model.pairwise_distance_vectors(vectores, vectores)

In [75]:
from sklearn.manifold import MDS
dim = 3
mds = MDS(n_components=dim, max_iter=6000, eps=0.001, random_state=0,
                   dissimilarity="precomputed", n_jobs=-1)
Y = mds.fit_transform(distancias)
Y.shape
df_to_plot.columns

Index(['user_id', 'user_name', 'name', 'description', 'url', 'created_at',
       'geo_enabled', 'verified', 'profile_image_url',
       'profile_image_url_large', 'followers_count', 'friends_count',
       'listed_count', 'favourites_count', 'statuses_count', 'media_count',
       'pinned_tweet', 'tweets_scraped', 'tweets_scraped_at', 'scrapped_at',
       'ignore', 'vector_user', 'n_tweets', 'partido', 'partido_original'],
      dtype='object')

In [88]:
plotData = pd.concat(
    [df_to_plot, pd.DataFrame(data = Y, index = df_to_plot.index, columns=['x','y','z'])],
    axis=1
)
# plotData = plotData[plotData.partido_original.notna()]

traces = []

for cname in plotData.partido.unique():
    df = plotData[plotData.partido==cname]
    traces.append(go.Scatter3d(
        name=cname,
        x=df.x,
        y=df.y,
        z=df.z,
        hovertext = "<b style=\"max-width:200px\">"+df.name+"</b><br>"+
                    df.description+'<br>'+
                    df.partido,
        hoverinfo='text',
        mode='markers',
        marker={
            'size':10
        }
    ))

fig = go.Figure(data=traces)
fig.update_layout(
#     autosize=False,
#     width=800,
    height=1050
)

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
], style={'height': '100vh'})
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



I0805 20:53:35.070051 16056 dash.py:1626] Dash is running on http://127.0.0.1:8050/



I0805 20:53:35.092053 16056 dash.py:1628]  Warning: This is a development server. Do not use app.run_server


 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production W

I0805 20:53:35.120051 16056 dash.py:1631]  in production, use a production WSGI server like gunicorn instead.



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
